In [1]:
# 运行该段代码以防止在训练过程中内核挂掉
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

## 以下是 Task1 和 Task3 的主要代码

In [12]:
import math
from argparse import ArgumentParser
from itertools import permutations

import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
from torch import nn
import torch.nn.functional as F

# 实现一个基本的 Transformer 模块
class Block(nn.Module):
    """
    Causal transformer block
    """

    def __init__(self, dim, num_heads):
        super().__init__()
        self.ln_1 = nn.LayerNorm(dim)
        self.ln_2 = nn.LayerNorm(dim)
        self.attn = nn.MultiheadAttention(dim, num_heads)
        self.mlp = nn.Sequential(
            nn.Linear(dim, dim * 4),
            nn.GELU(),
            nn.Linear(dim * 4, dim),
        )

    def forward(self, x):
        attn_mask = torch.full(
            (len(x), len(x)), -float("Inf"), device=x.device, dtype=x.dtype
        )
        attn_mask = torch.triu(attn_mask, diagonal=1)

        x = self.ln_1(x)
        a, _ = self.attn(x, x, x, attn_mask=attn_mask, need_weights=False)
        x = x + a
        m = self.mlp(self.ln_2(x))
        x = x + m
        return x


class Decoder(nn.Module):
    """
    Causal Transformer decoder
    """

    def __init__(self, dim=128, num_layers=2, num_heads=4, num_tokens=41, seq_len=5):
        super().__init__()
        self.token_embeddings = nn.Embedding(num_tokens, dim)
        self.position_embeddings = nn.Embedding(seq_len, dim)
        self.layers = nn.ModuleList()
        for _ in range(num_layers):
            self.layers.append(Block(dim, num_heads))

        self.ln_f = nn.LayerNorm(dim)
        self.head = nn.Linear(dim, num_tokens, bias=False)

    def forward(self, x):
        h = self.token_embeddings(x)
        positions = torch.arange(x.shape[0], device=x.device).unsqueeze(-1)
        h = h + self.position_embeddings(positions).expand_as(h)
        for layer in self.layers:
            h = layer(h)

        h = self.ln_f(h)
        logits = self.head(h)
        return logits

In [ ]:
# 生成二元加法运算 mod p 的数据集
def binary_plus_mod_p_data(p, eq_token, op_token):
    """
    x ◦ y = x + y (mod p) for 0 ≤ x < p, 0 ≤ y < p
    """
    x = torch.arange(p)
    y = torch.arange(p)
    x, y = torch.cartesian_prod(x, y).T

    eq = torch.ones_like(x) * eq_token
    op = torch.ones_like(x) * op_token
    result = (x + y) % p
    
    return torch.stack([x, op, y, eq, result])

# 生成二元除法运算 mod p 的数据集
def division_mod_p_data(p, eq_token, op_token):
    """
    x◦y = x/y (mod p) for 0 ≤ x < p, 0 < y < p
    """
    x = torch.arange(p)
    y = torch.arange(1, p)
    x, y = torch.cartesian_prod(x, y).T

    eq = torch.ones_like(x) * eq_token
    op = torch.ones_like(x) * op_token
    
    # 由于取模意义下的除法可表示为乘上其的逆，故此处采用乘法进行操作
    result = (x * y) % p   

    return torch.stack([x, op, y, eq, result])

In [15]:
def main(args):
    torch.manual_seed(0)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


    eq_token = args.p
    op_token = args.p + 1

    model = Decoder(
        dim=128, num_layers=2, num_heads=4, num_tokens=args.p + 2, seq_len=5
    ).to(device)

    # 生成数据
    data = division_mod_p_data(args.p, eq_token, op_token)
    train_ratio = 0.5               # 训练集比例,通过调整生成不同 training fraction 的结果
    split = int(data.shape[1] * train_ratio)
    idx_ = torch.randperm(data.shape[1])
    train_idx, valid_idx = idx_[:split],idx_[split:]
    train_data, valid_data = data[:, train_idx], data[:, valid_idx]

    # For most experiments we used AdamW optimizer with learning rate 10−3,
    # weight decay 1, β1 = 0.9, β2 = 0.98
    optimizer = getattr(torch.optim, args.optimizer)(
        model.parameters(),
        lr = args.lr,
        weight_decay = args.weight_decay,
        betas = (args.beta1, args.beta2),
    )

    #  linear learning rate warmup over the first 10 updates
    scheduler = torch.optim.lr_scheduler.LambdaLR(
        optimizer, lambda update: 1 if update > 10 else update / 10
    )

    steps_per_epoch = math.ceil(train_data.shape[1] / args.batch_size)

    train_acc, val_acc, train_loss, val_loss = [], [], [], []

    for e in tqdm(range(int(args.budget) // steps_per_epoch)):

        # randomly shuffle train data
        train_data = train_data[:, torch.randperm(train_data.shape[1])]

        for data, is_train in [(train_data, True), (valid_data, False)]:

            model.train(is_train)
            total_loss = 0
            total_acc = 0

            # torch.split faster than dataloader with tensor
            dl = torch.split(data, args.batch_size, dim=1)
            for input in dl:
                input = input.to(device)

                with torch.set_grad_enabled(is_train):
                    logits = model(input[:-1])
                    # calculate loss only on the answer part of the equation (last element
                    loss = F.cross_entropy(logits[-1], input[-1])
                    total_loss += loss.item() * input.shape[-1]

                if is_train:
                    model.zero_grad()
                    loss.backward()
                    optimizer.step()
                    scheduler.step()

                acc = (logits[-1].argmax(-1) == input[-1]).float().mean()
                total_acc += acc.item() * input.shape[-1]

            if is_train:
                train_acc.append(total_acc / train_data.shape[-1])
                train_loss.append(total_loss / train_data.shape[-1])
            else:
                val_acc.append(total_acc / valid_data.shape[-1])
                val_loss.append(total_loss / valid_data.shape[-1])

        if (e + 1) % 100 == 0:
            steps = torch.arange(len(train_acc)).numpy() * steps_per_epoch
            plt.plot(steps, train_acc, label="train")
            plt.plot(steps, val_acc, label="val")
            plt.legend()
            plt.title("Modular Division (training on 50% of data)")
            plt.xlabel("Optimization Steps")
            plt.ylabel("Accuracy")
            plt.xscale("log", base=10)
            
            # 在以下代码中，修改为自己的图像存储路径
            plt.savefig("figures-Task3/acc_weight_decay.png", dpi=150)
            plt.close()

            plt.plot(steps, train_loss, label="train")
            plt.plot(steps, val_loss, label="val")
            plt.legend()
            plt.title("Modular Division (training on 50% of data) ")
            plt.xlabel("Optimization Steps")
            plt.ylabel("Loss")
            plt.xscale("log", base=10)
            plt.savefig("figures-Task3/loss_weight_decay.png", dpi=150)
            plt.close()

if __name__ == "__main__":
    parser = ArgumentParser()
    parser.add_argument("--p", type = int, default = 41)
    parser.add_argument("--budget", type = int, default = 3e4)
    
    # 可以通过调整此处的默认参数以比较不同的 BatchSize
    parser.add_argument("--batch_size", type = int, default = 64)
    
    # 可以通过调整此处的默认参数以比较不同的学习率
    parser.add_argument("--lr", type = float, default = 1e-3)
    
    parser.add_argument("--beta1", type = float, default = 0.9)
    parser.add_argument("--beta2", type = float, default = 0.98)
    
    # 可以通过调整此处的默认参数以比较不同的 Weight decay 默认设置为 1
    parser.add_argument("--weight_decay", type = float, default = 1)
    
    # 可以通过调整此处的默认参数以比较不同的优化器
    parser.add_argument("--optimizer", default = "AdamW")
    
    # 可以通过添加以下代码，以实现 dropout
    # parser.add_argument("--dropout_prob", type=float, default=0.1)
    
    args = parser.parse_args(args = [])
    # 注意：如果此处出现代码报错，可以修改如下
    # args = paraser.parse_args()
    
    main(args)

100%|██████████████████████████████████████████████████████████████████████████████| 2307/2307 [25:44<00:00,  1.49it/s]


## 以下是 Task2 的主要代码

In [ ]:
class DecoderLSTM(nn.Module):
    """
    LSTM-based decoder
    """

    def __init__(self, dim=128, num_layers=2, num_tokens=97, seq_len=5):
        super().__init__()
        self.token_embeddings = nn.Embedding(num_tokens, dim)
        self.position_embeddings = nn.Embedding(seq_len, dim)

        # LSTM Layer
        self.lstm = nn.LSTM(dim, dim, num_layers, batch_first=True)

        # Final output layer
        self.head = nn.Linear(dim, num_tokens, bias=False)

    def forward(self, x):
        # Embedding + positional encoding
        h = self.token_embeddings(x)
        positions = torch.arange(x.shape[1], device=x.device).unsqueeze(-1)  # (batch_size, seq_len)
        h = h + self.position_embeddings(positions).expand_as(h)

        # Pass through LSTM
        h, _ = self.lstm(h)

        # Get logits for each token in the sequence
        logits = self.head(h)
        return logits

In [ ]:
class MLP(nn.Module):
    """
    MLP model for modular division.
    """

    def __init__(self, dim=128, num_layers=2, num_tokens=97, seq_len=5):
        super().__init__()
        self.token_embeddings = nn.Embedding(num_tokens, dim)
        self.position_embeddings = nn.Embedding(seq_len, dim)
        
        # Create the layers for the MLP
        mlp_layers = []
        for _ in range(num_layers):
            mlp_layers.append(nn.Linear(dim, dim))
            mlp_layers.append(nn.GELU())
        self.mlp = nn.Sequential(*mlp_layers)

        self.head = nn.Linear(dim, num_tokens)

    def forward(self, x):
        h = self.token_embeddings(x)
        positions = torch.arange(x.shape[0], device=x.device).unsqueeze(-1)
        h = h + self.position_embeddings(positions).expand_as(h)
        
        h = self.mlp(h)
        logits = self.head(h)
        return logits

In [ ]:
def main(args):
    torch.manual_seed(0)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    eq_token = args.p
    op_token = args.p + 1

    # Use LSTM-based Decoder
    model = DecoderLSTM(dim=128, num_layers=2, num_tokens=args.p + 2, seq_len=5).to(device)

    # Prepare data
    data = division_mod_p_data(args.p, eq_token, op_token)
    train_idx, valid_idx = torch.randperm(data.shape[1]).split(data.shape[1] // 2)
    train_data, valid_data = data[:, train_idx], data[:, valid_idx]

    optimizer = getattr(torch.optim, args.optimizer)(
        model.parameters(),
        lr=args.lr,
        weight_decay=args.weight_decay,
        betas=(args.beta1, args.beta2),
    )

    scheduler = torch.optim.lr_scheduler.LambdaLR(
        optimizer, lambda update: 1 if update > 10 else update / 10
    )

    steps_per_epoch = math.ceil(train_data.shape[1] / args.batch_size)

    train_acc, val_acc, train_loss, val_loss = [], [], [], []

    for e in tqdm(range(int(args.budget) // steps_per_epoch)):

        # randomly shuffle train data
        train_data = train_data[:, torch.randperm(train_data.shape[1])]

        for data, is_train in [(train_data, True), (valid_data, False)]:

            model.train(is_train)
            total_loss = 0
            total_acc = 0

            dl = torch.split(data, args.batch_size, dim=1)
            for input in dl:
                input = input.to(device)

                with torch.set_grad_enabled(is_train):
                    logits = model(input[:-1])
                    loss = F.cross_entropy(logits[-1], input[-1])
                    total_loss += loss.item() * input.shape[-1]

                if is_train:
                    model.zero_grad()
                    loss.backward()
                    optimizer.step()
                    scheduler.step()

                acc = (logits[-1].argmax(-1) == input[-1]).float().mean()
                total_acc += acc.item() * input.shape[-1]

            if is_train:
                train_acc.append(total_acc / train_data.shape[-1])
                train_loss.append(total_loss / train_data.shape[-1])
            else:
                val_acc.append(total_acc / valid_data.shape[-1])
                val_loss.append(total_loss / valid_data.shape[-1])

        if (e + 1) % 100 == 0:
            steps = torch.arange(len(train_acc)).numpy() * steps_per_epoch
            plt.plot(steps, train_acc, label="train")
            plt.plot(steps, val_acc, label="val")
            plt.legend()
            plt.title("Modular Division (LSTM-based Model)")
            plt.xlabel("Optimization Steps")
            plt.ylabel("Accuracy")
            plt.xscale("log", base=10)
            plt.savefig("figures/acc.png", dpi=150)
            plt.close()

            plt.plot(steps, train_loss, label="train")
            plt.plot(steps, val_loss, label="val")
            plt.legend()
            plt.title("Modular Division (LSTM-based Model)")
            plt.xlabel("Optimization Steps")
            plt.ylabel("Loss")
            plt.xscale("log", base=10)
            plt.savefig("figures/loss.png", dpi=150)
            plt.close()


if __name__ == "__main__":
    parser = ArgumentParser()
    parser.add_argument("--p", type=int, default=97)
    parser.add_argument("--budget", type=int, default=3e5)
    parser.add_argument("--batch_size", type=int, default=512)
    parser.add_argument("--lr", type=float, default=1e-3)
    parser.add_argument("--beta1", type=float, default=0.9)
    parser.add_argument("--beta2", type=float, default=0.98)
    parser.add_argument("--weight_decay", type=float, default=0)
    parser.add_argument("--optimizer", default="Adam")
    args = parser.parse_args(args = [])
    main(args)

## 以下是 Task4 的主要代码

In [3]:
# "由于上述代码中调用的 torch.cartesian_prod 函数仅支持计算两个张量的笛卡尔积，
#  故我们首先实现一个函数用于计算多个张量的笛卡尔积,为防止递归调用 torch.cartesian_prod" 
#  带来的内存开销,此处选择 网格数据 + 平展 的方式生成笛卡尔积." 

def cartesian_prod_multi(tensors_list):
    
    # 确保至少有两个张量用于计算笛卡尔积
    if len(tensors_list) < 2:
        raise ValueError("At least two tensors are required for Cartesian product.")
    
    # 使用 torch.meshgrid 创建网格
    mesh = torch.meshgrid(*tensors_list, indexing='ij')
    
    # 将每个网格展平并堆叠在一起
    flattened = [m.flatten() for m in mesh]
    
    # 转置以获得正确的顺序，并将结果转换为单个张量
    result = torch.stack(flattened, dim=1)
    
    return result.T

In [4]:
# 对多元运算问题,生成训练和测试数据
def multivariate_plus_mod_p_data(p , eq_token , op_token ,k):
    """
    x_1 ◦ x_2 ◦ ... ◦ x_k = x_1 + x_2 + ... + x_k (mod p) for 0 ≤ x_i < p (i = 1,2,...,k)
    """
    # 生成 k 个向量的笛卡尔积
    tensor_list,stack_list = [],[]
    for _ in range(k):
        tensor_list.append(torch.arange(p))
    combine_tensor = cartesian_prod_multi(tensor_list)

    # 生成运算符的编码向量
    eq = torch.ones((combine_tensor.shape[1],)) * eq_token
    op = torch.ones((combine_tensor.shape[1],)) * op_token
    
    # 生成运算结果向量
    result = torch.sum(combine_tensor,axis = 0) % p
    
    # 将所有编码向量按顺序组合
    for i in range(combine_tensor.shape[0]):
        stack_list.append(combine_tensor[i])
        stack_list.append(op)
    stack_list.append(eq)
    stack_list.append(result)
    
    return torch.stack(stack_list).to(torch.long)

In [ ]:
# 与 Task1-Task3 的代码基本相同，只是添加了一个参数 args.k 表示多元运算的维度
def main(args):
    torch.manual_seed(0)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


    eq_token = args.p
    op_token = args.p + 1

    model = Decoder(
        dim=128, num_layers=2, num_heads=4, num_tokens=args.p + 2, seq_len= 2 * args.k + 1
    ).to(device)

    data = multivariate_plus_mod_p_data(args.p, eq_token, op_token,args.k)
    train_ratio = 0.8               # 训练集比例,通过调整生成不同 training fraction 的结果
    split = int(data.shape[1] * train_ratio)
    idx_ = torch.randperm(data.shape[1])
    train_idx, valid_idx = idx_[:split],idx_[split:]
    train_data, valid_data = data[:, train_idx], data[:, valid_idx]

    # For most experiments we used AdamW optimizer with learning rate 10−3,
    # weight decay 1, β1 = 0.9, β2 = 0.98
    optimizer = getattr(torch.optim, args.optimizer)(
        model.parameters(),
        lr = args.lr,
        weight_decay = args.weight_decay,
        betas = (args.beta1, args.beta2),
    )

    #  linear learning rate warmup over the first 10 updates
    scheduler = torch.optim.lr_scheduler.LambdaLR(
        optimizer, lambda update: 1 if update > 10 else update / 10
    )

    steps_per_epoch = math.ceil(train_data.shape[1] / args.batch_size)

    train_acc, val_acc, train_loss, val_loss = [], [], [], []

    for e in tqdm(range(int(args.budget) // steps_per_epoch)):

        # randomly shuffle train data
        train_data = train_data[:, torch.randperm(train_data.shape[1])]

        for data, is_train in [(train_data, True), (valid_data, False)]:

            model.train(is_train)
            total_loss = 0
            total_acc = 0

            # torch.split faster than dataloader with tensor
            dl = torch.split(data, args.batch_size, dim=1)
            for input in dl:
                input = input.to(device)

                with torch.set_grad_enabled(is_train):
                    logits = model(input[:-1])
                    # calculate loss only on the answer part of the equation (last element
                    loss = F.cross_entropy(logits[-1], input[-1])
                    total_loss += loss.item() * input.shape[-1]

                if is_train:
                    model.zero_grad()
                    loss.backward()
                    optimizer.step()
                    scheduler.step()

                acc = (logits[-1].argmax(-1) == input[-1]).float().mean()
                total_acc += acc.item() * input.shape[-1]

            if is_train:
                train_acc.append(total_acc / train_data.shape[-1])
                train_loss.append(total_loss / train_data.shape[-1])
            else:
                val_acc.append(total_acc / valid_data.shape[-1])
                val_loss.append(total_loss / valid_data.shape[-1])

        if (e + 1) % 10 == 0:
            steps = torch.arange(len(train_acc)).numpy() * steps_per_epoch
            plt.plot(steps, train_acc, label="train")
            plt.plot(steps, val_acc, label="val")
            plt.legend()
            plt.title("Modular Division (training on 80% of data) with p = 41 and K = 3")
            plt.xlabel("Optimization Steps")
            plt.ylabel("Accuracy")
            plt.xscale("log", base=10)
            plt.savefig("figures-Task4/acc_k_3_80.png", dpi=150)  # 修改图像的存储路径
            plt.close()

            plt.plot(steps, train_loss, label="train")
            plt.plot(steps, val_loss, label="val")
            plt.legend()
            plt.title("Modular Division (training on 80% of data) with p = 41 and K = 3")
            plt.xlabel("Optimization Steps")
            plt.ylabel("Loss")
            plt.xscale("log", base=10)
            plt.savefig("figures-Task4/loss_k_3_80.png", dpi=150)
            plt.close()

if __name__ == "__main__":
    parser = ArgumentParser()
    parser.add_argument("--p", type = int, default = 41)
    parser.add_argument("--budget", type = int, default = 3e5)
    parser.add_argument("--batch_size", type = int, default = 32)
    parser.add_argument("--lr", type = float, default = 1e-3)
    parser.add_argument("--beta1", type = float, default = 0.9)
    parser.add_argument("--beta2", type = float, default = 0.98)
    parser.add_argument("--weight_decay", type = float, default = 1)
    parser.add_argument("--optimizer", default = "AdamW")
    
    # 多元运算的维度，默认设定为 k = 3，可以尝试更大的 K,但会受到内存的限制。
    parser.add_argument("--k", type = int, default = 3)
    
    args = parser.parse_args(args = [])
    main(args)

 11%|████████▉                                                                     | 20/174 [35:43<9:01:29, 210.97s/it]